Testing aws api here and LLM


In [16]:
!pip install boto3 matplotlib

In [17]:
import boto3

# Create a session with your SSO profile
session = boto3.Session(profile_name="myisb01_IsbUsersPS-064566626462")

# Then create the Bedrock client from that session
bedrock = session.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)


In [18]:
# Simulated OCR output of student homework
homework_text = """
Describe an event which you did not look forward to attending but

turned out to exciting.

As a young teenager growing up in Singapore, I often found myself dreading certain
events or activities that my family and friends would plan. Whether it was a family
gathering, a school field trip, or a friend's birthday party, there were times when I just
did not feel like going. However, one particular event stands out in my memory as an
occasion where I surprised myself by having an unexpectedly great time.

It was a warm Saturday afternoon, and my mom had told me that we were going to
visit my great aunt who lived in a different part of the city. I was not particularly close
to my great aunt, and I did not really know what to expect from the visit. In fact, I was
feeling a bit grumpy and resentful about having to spend my weekend doing
something that I was not interested in. As we made our way to her house, I kept
thinking about all the other things I could have been doing instead.

When we arrived, my great aunt greeted us with a warm smile and a big hug. She
was a kind and gentle woman, and I immediately felt at ease in her company. As we
settled into her living room, she offered us some snacks and began to tell us stories
about her life. I was surprised to find that I was actually interested in what she had to
say, and I listened attentively as she shared her experiences growing up in
Singapore during a different era.

After a while, my greatest of aunt suggested that we take a walks around her
neighbourhood. I wasn't not sure what to expect, but I agreed to join her and my mom.
As we walked, my great aunt pointed out different landmarks and shared more
stories about her past. I found myself enjoying the walk, taking in the sights and

sounds of the neighborhood, and feeling grateful to be spending time with my family.
"""

In [ ]:
import json

prompt_classify = f"""
<role>
You are an English tutor. 
A student submitted the following homework:
</role>

<homework>{homework_text}</homework>

<instructions>
Analyze the student's work and identify flaws in the homework regarding each of these categories, envelope within <analysis> tags:
- Grammar
- Vocabulary
- Writing
- Spelling
- Punctuation

Provide a strength index (1 to 5), in JSON format, enveloped in tags <attribute-scores>:
<example>
{{
  "Grammar": 5.00,
  "Vocabulary": 3.14,
  "Writing": 2.23,
  "Spelling": 1.19,
  "Punctuation": 4.56
}}
</example>
</instructions>
"""

response = bedrock.invoke_model(
    modelId="anthropic.claude-v2:1",
    body=json.dumps({
        "prompt": f"\n\nHuman: {prompt_classify}\n\nAssistant:",
        "max_tokens_to_sample": 300,
        "temperature": 0.3,
        "stop_sequences": ["\n\nHuman:"]
    })
)

result = json.loads(response["body"].read())
weakness_summary = result["completion"]
print("Weakness Classification:\n", weakness_summary)

Weakness Classification:
  <analysis>

Grammar: 
- Inconsistent verb tense usage - "I was feeling" vs "I kept thinking"
- Run-on sentence: "As we made our way to her house, I kept thinking about all the other things I could have been doing instead." This should be two separate sentences.
- Missing word in sentence: "I wasn't not sure what to expect" should be "I was not sure what to expect"

Vocabulary:
- Good vocabulary usage overall

Writing:
- Good sequence of events and details
- Could provide more insight into thoughts and feelings at key moments

Spelling:  
- Incorrect spelling of "greatest" - should be "great"
- Incorrect spelling of "neighbourhood" - should be "neighborhood"

Punctuation:
- Missing commas in compound sentences, for example: "As we walked my great aunt pointed out different landmarks and shared more stories about her past."
- Run-on sentence lacks proper punctuation
</analysis>

<attribute-scores>
{
  "Grammar": 3.50,
  "Vocabulary": 4.00,
  "Writing": 3.50,
  

In [ ]:
    # Parse the response
    result = json.loads(response["body"].read())
    weakness_summary = result["content"][0]["text"]

{'Grammar': 3.5, 'Vocabulary': 4.0, 'Writing': 3.5, 'Spelling': 3.0, 'Punctuation': 3.0}


In [ ]:
    # Parse the response
    result2 = json.loads(response2["body"].read())
    exercises = result2["content"][0]["text"]

    # Convert the exercises string to JSON
    exercises_json = json.loads(exercises.strip())
    print(json.dumps(exercises_json, indent=2))

 Here are 3 short exercises to target the student's weaknesses:

[
  {
    "Exercise": "Rewrite the following sentence correcting the verb tense: 'I was feeling nervous as I walked up to the door.'",
    "Suggested_Answer": "I felt nervous as I walked up to the door." 
  },
  {
    "Exercise": "Break the following run-on sentence into two separate sentences with proper punctuation: 'As we made our way to her house, I kept thinking about all the other things I could have been doing instead.'",
    "Suggested_Answer": "As we made our way to her house, I thought about all the other things I could have been doing. I kept thinking I would rather be somewhere else."
  },
  {
    "Exercise": "Correct the spelling mistakes in the following sentence: 'We walked through the neighbourhood my great aunt grew up in.'", 
    "Suggested_Answer": "We walked through the neighborhood my great aunt grew up in."
  }
]
